In [1]:
import imutils
import cv2
import numpy as np

In [2]:
# 3.1.3 & 3.1.5 & 3.2.1
# lower and upper boundaries for HSV thresholds
Green_Lower = (29, 86, 6)
Green_Upper = (64, 255, 255)
Blue_Lower = (100, 150, 0)
Blue_Upper = (140, 255, 255)
Red_Lower = (0, 70, 50)
Red_Upper = (10, 255, 255)


def prep(frame, colour):
    frame = imutils.resize(frame, width=600)
    kernal = np.ones((5, 5), np.uint8) 
    ## Todo 3.1.3 gaussian blur
    blurred = cv2.GaussianBlur(frame,(11,11),0)
    hsv = cv2.cvtColor(blurred, cv2.COLOR_BGR2HSV)
    
    # Colour mask
    if colour == "green":
        mask = cv2.inRange(hsv, Lower, Upper)
    elif colour == "Blue":
        mask = cv2.inRange(hsv, Blue_Lower, Blue_Upper)
    else:
        mask = cv2.inRange(hsv, Red_Lower, Red_Upper)
    
    ## Todo 3.1.3 gaussian blur
    mask = cv2.erode(mask, kernal, 1) 
    mask = cv2.dilate(mask, kernal, 1) 

    
    return mask, frame

frame = cv2.imread('image.png')
mask,frame = prep(frame, "Blue")


In [ ]:
# 3.1.3
cv2.imshow("Image",frame)
cv2.imshow("Processed Image",mask)
cv2.waitKey(0) & 0xFF
cv2.destroyAllWindows()

In [3]:
# 3.1.5 & 3.2.1
### Find contours and display
def find_cnts(mask):
    cnts = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    return cnts

In [ ]:
# 3.1.5

cnts = find_cnts(mask)
c = max(cnts, key=cv2.contourArea)
((x, y), radius) = cv2.minEnclosingCircle(c) ## A different contour?
        
# Find center of contour using moments in opencvq
M = cv2.moments(c)
center = (int(M["m10"] / M["m00"]), int(M["m01"] / M["m00"]))

#circle
frame1 = frame
cv2.circle(frame1, (int(x), int(y)), int(radius),(0, 0, 255), 2)
cv2.circle(frame1, center, 5, (0, 0, 255), -1)  
cv2.imshow("Circle", frame1)


## ToDo 3.1.5 
#  1. Bounding rectangle
frame2 = frame
x,y,w,h = cv2.boundingRect(c)
cv2.rectangle(frame2,(x,y),(x+w,y+h),(0,255,0),2)
cv2.imshow("Bounding rectangle", frame2)

#  1.1 Rotated rectangle
frame3 = frame
rect = cv2.minAreaRect(c)
box = cv2.boxPoints(rect)
box = np.intp(box)
cv2.drawContours(frame3,[box],0,(0,0,255),2)
cv2.imshow("Rotated rectangle", frame3)

#  2 Convex Hull
frame4 = frame
hull = cv2.convexHull(c)
cv2.drawContours(frame4,[hull],0,(255,255,0),2)
cv2.imshow("Convex Hull", frame4)

cv2.waitKey(0) & 0xFF
cv2.destroyAllWindows()

In [4]:
# 3.2.1
from imutils.video import VideoStream
import time

vs = cv2.VideoCapture("test_video.avi")
time.sleep(2.0)

frame_rate = 30
prev = 0

while True :
    time_elapsed = time.time() - prev

    
    if time_elapsed > 1./frame_rate:
        frame = vs.read()
        frame = frame[1]
        if frame is None:
            break
        cv2.imshow("Frame", frame)
        mask, frame = prep(frame, "Blue")
        cnts = find_cnts(mask)
        c = max(cnts, key=cv2.contourArea)
        frame1 = frame
        hull = cv2.convexHull(c)
        cv2.drawContours(frame1,[hull],0,(0,255,0),2)
        cv2.imshow("Convex Hull", frame1)
        key = cv2.waitKey(1) & 0xFF
        if key == ord("q"):
            break
        prev = time.time()
            
cv2.destroyAllWindows()   

In [ ]:
# 3.3

import sys
import rclpy
from rclpy.node import Node
import imutils
import numpy as np
import cv2
from sensor_msgs.msg import Image, CameraInfo
from cv_bridge import CvBridge
#print(sys.path)

# lower and upper boundaries for HSV thresholds (Blue)
Lower = (100, 150, 0)
Upper = (140, 255, 255)

bridge = CvBridge()

class Get_Images(Node):

    def __init__(self):
        # Initialize the node
        super().__init__('Image_Subscriber')
        # Initialize the subscriber
        self.subscription_ = self.create_subscription(Image,'/camera', self.listener_callback,10)
        self.subscription_  # prevent unused variable warning
        timer_period = 0.1  # seconds
        self.timer = self.create_timer(timer_period, self.timer_callback)
        self.K = True

    def listener_callback(self, msg):
        height = msg.height
        width = msg.width
        channel = msg.step//msg.width
        #frame = np.reshape(msg.data, (height, width, channel))
        self.frame = bridge.imgmsg_to_cv2(msg, desired_encoding='bgr8')
        #self.get_logger().info("Image Received")
        return self.frame
    
    def timer_callback(self):
        if self.K == True:
            cv2.imshow("Tracking",self.frame)
            mask, frame = self.prep(self.frame)
            cnts = self.find_cnts(mask)
            c = max(cnts, key=cv2.contourArea)
            frame1 = frame
            hull = cv2.convexHull(c)
            cv2.drawContours(frame1,[hull],0,(0,255,0),2)
            cv2.imshow("Convex Hull", frame1)
            key = cv2.waitKey(1) & 0xFF
            if key == ord("q"):
                self.get_logger().info('Closing stream window..')
                self.K = False
                cv2.destroyAllWindows()
        
           
    def stop_stream(self):
        self.get_logger().info('Stopping the stream ...')
        
    def prep(self, frame):
        frame = imutils.resize(frame, width=600)
        # kernal = np.ones((5, 5), np.uint8) 
        ## Todo 3.1.3 gaussian blur
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

        # Colour mask
        mask = cv2.inRange(hsv, Lower, Upper)

        ## Todo 3.1.3 gaussian blur
        # mask = cv2.erode(mask, kernal, 1) 
        # mask = cv2.dilate(mask, kernal, 1)
        
        return mask, frame
    
    ### Find contours and display
    def find_cnts(self, mask):
        cnts = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
        cnts = imutils.grab_contours(cnts)
        return cnts
        

try:
    
    rclpy.init(args=None)
    image_subscriber = Get_Images()
    rclpy.spin(image_subscriber)

except KeyboardInterrupt:
    # executes on keyboard kernal interrupt with double pressing button "i"
    image_subscriber.stop_stream()
    image_subscriber.destroy_node()
    rclpy.shutdown()

[INFO] [1700642526.977402656] [Image_Subscriber]: Closing stream window..
